# Module 03: Django Admin Interface

**Estimated Time:** 1.5 hours  
**Difficulty:** ⭐

---

## Prerequisites

Before starting this module, ensure you've completed:

- ✅ **Module 00: Setup & Introduction**
- ✅ **Module 01: Django Basics**
- ✅ **Module 02: Models & Databases**
- ✅ **Post and Category models created**

---

## Learning Objectives

By the end of this module, you will:

- ✅ Create a superuser account
- ✅ Access and navigate Django admin interface
- ✅ Register models with admin
- ✅ Customize admin displays and list views
- ✅ Add search functionality
- ✅ Implement filters and ordering
- ✅ Create custom admin actions

---

## Prerequisites

Before starting this module, make sure you've completed:
- ✅ Module 00: Setup & Introduction
- ✅ Module 01: Django Basics & First Project
- ✅ Module 02: Models & Databases

You should have:
- A Django project (`myblog`) with the `blog` app
- Models created (Post, Category)
- Database migrations applied

## 1. Introduction to Django Admin

One of Django's most powerful features is its **automatic admin interface**. It's a production-ready interface that reads metadata from your models to provide a quick, model-centric interface where trusted users can manage content.

### Why Use Django Admin?

- **Automatic**: Generated from your models
- **Production-ready**: Used by real sites for content management
- **Customizable**: Extensive customization options
- **Saves time**: No need to build your own admin interface
- **Built-in features**: Authentication, permissions, CRUD operations

### Who Uses It?

- **Content editors**: Add/edit blog posts, products, etc.
- **Administrators**: Manage users and permissions
- **Developers**: Quick data management during development

## 2. Setup Paths

Let's set up our working environment.

In [ ]:
import os
import subprocess
from pathlib import Path

# Setup paths
notebook_dir = Path.cwd()
project_path = notebook_dir.parent / "projects" / "myblog"
blog_app_path = project_path / "blog"
admin_file = blog_app_path / "admin.py"

print(f"Project path: {project_path}")
print(f"Blog app path: {blog_app_path}")
print(f"Admin file: {admin_file}")
print(f"Project exists: {project_path.exists()}")

## 3. Creating a Superuser

To access the admin interface, you need a user account with staff and superuser status. Let's create one.

### Creating Superuser via Command Line

**In a terminal**, run:
```bash
cd projects/myblog
python manage.py createsuperuser
```

You'll be prompted for:
- **Username**: Choose a username (e.g., admin)
- **Email**: Your email address (optional)
- **Password**: Choose a strong password (Django will validate it)

### Creating Superuser Programmatically (for automation)

In [ ]:
# Check if superuser already exists
# Note: This is for demonstration. In practice, use the command line.

check_user_script = """
from django.contrib.auth.models import User
import sys
import os

# Django setup
sys.path.insert(0, r'{}')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'myblog.settings')

import django
django.setup()

# Check for admin user
admin_exists = User.objects.filter(username='admin').exists()
print(f'Admin user exists: {admin_exists}')

if not admin_exists:
    print('To create admin user, run in terminal:')
    print('python manage.py createsuperuser')
else:
    admin = User.objects.get(username='admin')
    print(f'Username: {admin.username}')
    print(f'Email: {admin.email}')
    print(f'Is superuser: {admin.is_superuser}')
    print(f'Is staff: {admin.is_staff}')
""".format(
    project_path
)

# Save and run the script
check_script_path = project_path / "check_admin.py"
with open(check_script_path, "w") as f:
    f.write(check_user_script)

result = subprocess.run(
    ["python", str(check_script_path)], capture_output=True, text=True, cwd=project_path
)

print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

### User Model Fields

The Django User model has important fields:

| Field | Description |
|-------|-------------|
| `username` | Unique username |
| `email` | Email address |
| `password` | Hashed password |
| `first_name` | First name |
| `last_name` | Last name |
| `is_staff` | Can access admin interface |
| `is_superuser` | Has all permissions |
| `is_active` | Account is active |
| `date_joined` | Registration date |

## 4. Accessing the Admin Interface

Now that you have a superuser account, let's access the admin interface.

### Steps:

1. **Start the development server** (in a separate terminal):
   ```bash
   cd projects/myblog
   python manage.py runserver
   ```

2. **Open your browser** and visit:
   ```
   http://127.0.0.1:8000/admin/
   ```

3. **Login** with your superuser credentials

### What You'll See

The admin homepage shows:
- **Authentication and Authorization**: Users and Groups
- Your registered models (we'll add our blog models next)
- Links to add, change, and delete records

## 5. Registering Models with Admin

To make your models manageable through the admin interface, you need to register them.

### Basic Registration

In [ ]:
# Create basic admin registration
basic_admin_code = """from django.contrib import admin
from .models import Post, Category

# Basic registration
admin.site.register(Post)
admin.site.register(Category)
"""

# Write to admin.py
with open(admin_file, "w") as f:
    f.write(basic_admin_code)

print("✓ Models registered with admin (basic)")
print("\nNow visit http://127.0.0.1:8000/admin/ and you'll see Post and Category!")

## 6. Customizing the Admin Interface

Basic registration works, but Django admin is highly customizable. Let's create a better admin experience.

### Admin Class Options

The `ModelAdmin` class provides many customization options:

- **list_display**: Fields to show in the list view
- **list_filter**: Add filter sidebar
- **search_fields**: Enable search functionality
- **ordering**: Default ordering
- **prepopulated_fields**: Auto-fill fields based on others
- **date_hierarchy**: Date-based drill-down navigation
- **fields**: Customize form field layout
- **readonly_fields**: Make fields read-only
- **list_per_page**: Items per page

In [ ]:
# Create customized admin
custom_admin_code = '''from django.contrib import admin
from .models import Post, Category


@admin.register(Category)
class CategoryAdmin(admin.ModelAdmin):
    """Admin configuration for Category model"""
    
    list_display = ['name', 'slug', 'description']
    search_fields = ['name', 'description']
    prepopulated_fields = {'slug': ('name',)}
    ordering = ['name']


@admin.register(Post)
class PostAdmin(admin.ModelAdmin):
    """Admin configuration for Post model"""
    
    # List view customization
    list_display = ['title', 'author', 'status', 'publish_date', 'views', 'created_date']
    list_filter = ['status', 'created_date', 'publish_date', 'author']
    search_fields = ['title', 'content']
    prepopulated_fields = {'slug': ('title',)}
    date_hierarchy = 'publish_date'
    ordering = ['-publish_date']
    list_per_page = 20
    
    # Form customization
    fields = [
        'title',
        'slug',
        'author',
        'content',
        'categories',
        'status',
        'publish_date',
        'views',
    ]
    
    readonly_fields = ['views']
    
    # Filter by category (many-to-many)
    filter_horizontal = ['categories']
    
    # Custom methods for list display
    def get_queryset(self, request):
        """Optimize queryset with select_related"""
        qs = super().get_queryset(request)
        return qs.select_related('author')
'''

# Write customized admin
with open(admin_file, "w") as f:
    f.write(custom_admin_code)

print("✓ Admin interface customized!")
print("\nCustomizations added:")
print("  - List displays with multiple columns")
print("  - Search functionality")
print("  - Filters by status, date, and author")
print("  - Auto-populated slug fields")
print("  - Date hierarchy navigation")
print("  - Read-only fields")
print("  - Horizontal filter for categories")

### Understanding the Customizations

#### list_display
```python
list_display = ['title', 'author', 'status', 'publish_date']
```
Shows these fields as columns in the admin list view.

#### list_filter
```python
list_filter = ['status', 'created_date', 'author']
```
Adds a filter sidebar to narrow down results.

#### search_fields
```python
search_fields = ['title', 'content']
```
Enables search box that searches these fields.

#### prepopulated_fields
```python
prepopulated_fields = {'slug': ('title',)}
```
Automatically generates slug from title as you type.

#### date_hierarchy
```python
date_hierarchy = 'publish_date'
```
Adds date-based drill-down navigation.

#### readonly_fields
```python
readonly_fields = ['views']
```
Makes fields visible but not editable.

## 7. Adding Custom Display Methods

You can add custom methods to display computed or formatted data in the admin list view.

In [ ]:
# Enhanced admin with custom display methods
enhanced_admin_code = '''from django.contrib import admin
from django.utils.html import format_html
from django.utils import timezone
from .models import Post, Category


@admin.register(Category)
class CategoryAdmin(admin.ModelAdmin):
    """Admin configuration for Category model"""
    
    list_display = ['name', 'slug', 'post_count']
    search_fields = ['name', 'description']
    prepopulated_fields = {'slug': ('name',)}
    ordering = ['name']
    
    def post_count(self, obj):
        """Display number of posts in this category"""
        count = obj.posts.count()
        return f"{count} post{'s' if count != 1 else ''}"
    post_count.short_description = 'Posts'


@admin.register(Post)
class PostAdmin(admin.ModelAdmin):
    """Admin configuration for Post model"""
    
    list_display = [
        'title',
        'author',
        'status_badge',
        'publish_date',
        'views',
        'is_recent',
        'category_list',
    ]
    list_filter = ['status', 'created_date', 'publish_date', 'author', 'categories']
    search_fields = ['title', 'content']
    prepopulated_fields = {'slug': ('title',)}
    date_hierarchy = 'publish_date'
    ordering = ['-publish_date']
    list_per_page = 20
    filter_horizontal = ['categories']
    
    fields = [
        'title',
        'slug',
        'author',
        'content',
        'categories',
        'status',
        'publish_date',
        'views',
    ]
    
    readonly_fields = ['views']
    
    def status_badge(self, obj):
        """Display status with colored badge"""
        colors = {
            'published': 'green',
            'draft': 'orange',
        }
        color = colors.get(obj.status, 'gray')
        return format_html(
            '<span style="padding: 3px 10px; background-color: {}; color: white; '
            'border-radius: 3px; font-size: 11px;">{}</span>',
            color,
            obj.status.upper()
        )
    status_badge.short_description = 'Status'
    
    def is_recent(self, obj):
        """Check if post was published recently"""
        if obj.publish_date:
            delta = timezone.now() - obj.publish_date
            if delta.days <= 7:
                return format_html('<span style="color: green;">✓ Recent</span>')
        return format_html('<span style="color: gray;">-</span>')
    is_recent.short_description = 'Recent?'
    
    def category_list(self, obj):
        """Display categories as comma-separated list"""
        categories = obj.categories.all()
        if categories:
            return ", ".join([cat.name for cat in categories])
        return "-"
    category_list.short_description = 'Categories'
    
    def get_queryset(self, request):
        """Optimize queryset"""
        qs = super().get_queryset(request)
        return qs.select_related('author').prefetch_related('categories')
'''

# Write enhanced admin
with open(admin_file, "w") as f:
    f.write(enhanced_admin_code)

print("✓ Enhanced admin with custom display methods!")
print("\nNew features:")
print("  - Colored status badges")
print("  - Recent post indicator")
print("  - Category list display")
print("  - Post count in categories")
print("  - Optimized database queries")

## 8. Adding Custom Admin Actions

Admin actions allow you to perform bulk operations on selected items.

In [ ]:
# Add custom actions
actions_admin_code = '''from django.contrib import admin
from django.utils.html import format_html
from django.utils import timezone
from .models import Post, Category


@admin.register(Category)
class CategoryAdmin(admin.ModelAdmin):
    """Admin configuration for Category model"""
    
    list_display = ['name', 'slug', 'post_count']
    search_fields = ['name', 'description']
    prepopulated_fields = {'slug': ('name',)}
    ordering = ['name']
    
    def post_count(self, obj):
        count = obj.posts.count()
        return f"{count} post{'s' if count != 1 else ''}"
    post_count.short_description = 'Posts'


@admin.register(Post)
class PostAdmin(admin.ModelAdmin):
    """Admin configuration for Post model"""
    
    list_display = [
        'title',
        'author',
        'status_badge',
        'publish_date',
        'views',
        'is_recent',
    ]
    list_filter = ['status', 'created_date', 'publish_date', 'author', 'categories']
    search_fields = ['title', 'content']
    prepopulated_fields = {'slug': ('title',)}
    date_hierarchy = 'publish_date'
    ordering = ['-publish_date']
    list_per_page = 20
    filter_horizontal = ['categories']
    readonly_fields = ['views']
    
    # Custom actions
    actions = ['make_published', 'make_draft', 'reset_views']
    
    def status_badge(self, obj):
        colors = {'published': 'green', 'draft': 'orange'}
        color = colors.get(obj.status, 'gray')
        return format_html(
            '<span style="padding: 3px 10px; background-color: {}; color: white; '
            'border-radius: 3px; font-size: 11px;">{}</span>',
            color, obj.status.upper()
        )
    status_badge.short_description = 'Status'
    
    def is_recent(self, obj):
        if obj.publish_date:
            delta = timezone.now() - obj.publish_date
            if delta.days <= 7:
                return format_html('<span style="color: green;">✓ Recent</span>')
        return format_html('<span style="color: gray;">-</span>')
    is_recent.short_description = 'Recent?'
    
    # Custom Actions
    def make_published(self, request, queryset):
        """Bulk publish selected posts"""
        updated = queryset.update(status='published', publish_date=timezone.now())
        self.message_user(request, f'{updated} post(s) successfully published.')
    make_published.short_description = "Publish selected posts"
    
    def make_draft(self, request, queryset):
        """Bulk change selected posts to draft"""
        updated = queryset.update(status='draft')
        self.message_user(request, f'{updated} post(s) changed to draft.')
    make_draft.short_description = "Change to draft"
    
    def reset_views(self, request, queryset):
        """Reset view count to zero"""
        updated = queryset.update(views=0)
        self.message_user(request, f'Reset views for {updated} post(s).')
    reset_views.short_description = "Reset view count"
    
    def get_queryset(self, request):
        qs = super().get_queryset(request)
        return qs.select_related('author').prefetch_related('categories')
'''

# Write admin with actions
with open(admin_file, "w") as f:
    f.write(actions_admin_code)

print("✓ Custom admin actions added!")
print("\nAvailable actions:")
print("  1. Publish selected posts - Sets status to 'published' and updates publish_date")
print("  2. Change to draft - Sets status to 'draft'")
print("  3. Reset view count - Resets views to 0")
print("\nTo use: Select posts → Choose action from dropdown → Click 'Go'")

## 9. Advanced Admin Features

### Inlines

Inlines allow you to edit related objects on the same page. For example, if we had a Comment model, we could edit comments directly on the post edit page.

```python
class CommentInline(admin.TabularInline):
    model = Comment
    extra = 1

@admin.register(Post)
class PostAdmin(admin.ModelAdmin):
    inlines = [CommentInline]
```

### Fieldsets

Organize form fields into sections:

```python
fieldsets = [
    ('Basic Information', {
        'fields': ['title', 'slug', 'author']
    }),
    ('Content', {
        'fields': ['content', 'categories']
    }),
    ('Publishing', {
        'fields': ['status', 'publish_date'],
        'classes': ['collapse'],  # Collapsible section
    }),
]
```

### List Editable

Edit fields directly in the list view:

```python
list_editable = ['status']
```

## 10. Admin Site Customization

You can customize the admin site itself.

In [ ]:
# Read current admin file
with open(admin_file, "r") as f:
    current_admin = f.read()

# Add admin site customization at the top
site_customization = """# Admin site customization
admin.site.site_header = "My Blog Administration"
admin.site.site_title = "My Blog Admin"
admin.site.index_title = "Welcome to My Blog Admin Panel"

"""

# Add it after imports
final_admin = current_admin.replace(
    "from .models import Post, Category",
    "from .models import Post, Category\n\n" + site_customization,
)

with open(admin_file, "w") as f:
    f.write(final_admin)

print("✓ Admin site customized!")
print("\nChanges:")
print("  - Custom header: 'My Blog Administration'")
print("  - Custom title: 'My Blog Admin'")
print("  - Custom index title: 'Welcome to My Blog Admin Panel'")

## 11. Testing the Admin Interface

Let's verify our admin configuration is valid.

In [ ]:
# Check for any admin errors
result = subprocess.run(
    ["python", "manage.py", "check"], capture_output=True, text=True, cwd=project_path
)

print("System check:")
print(result.stdout)

if "no issues" in result.stdout.lower():
    print("\n✓ Admin configuration is valid!")
else:
    print("\n⚠ Check output above for any issues")

## 12. Summary of Admin Customizations

Here's a comprehensive summary of what we've customized:

### Category Admin
- ✅ List display: name, slug, post count
- ✅ Search by name and description
- ✅ Auto-populate slug from name
- ✅ Custom post count display

### Post Admin
- ✅ List display: title, author, status badge, date, views, recent indicator
- ✅ Filters: status, dates, author, categories
- ✅ Search: title and content
- ✅ Auto-populate slug
- ✅ Date hierarchy navigation
- ✅ Horizontal filter for categories
- ✅ Read-only views field
- ✅ Colored status badges
- ✅ Recent post indicator
- ✅ Custom actions: publish, draft, reset views
- ✅ Optimized queries

### Site-wide
- ✅ Custom site header, title, and index title

## 13. Best Practices

### Performance
1. **Use select_related and prefetch_related**
   ```python
   def get_queryset(self, request):
       qs = super().get_queryset(request)
       return qs.select_related('author').prefetch_related('categories')
   ```

2. **Limit list_per_page** for large datasets
   ```python
   list_per_page = 25
   ```

### User Experience
1. **Use meaningful short_description** for custom methods
2. **Add help_text** to model fields
3. **Use list_editable** for quick edits
4. **Add date_hierarchy** for time-based data

### Security
1. **Never expose admin in production** without strong passwords
2. **Use HTTPS** in production
3. **Limit admin access** to trusted users
4. **Consider custom admin URL** in production

### Organization
1. **Keep admin.py organized** by model
2. **Use docstrings** for complex customizations
3. **Group related fieldsets**
4. **Comment custom actions**

## 14. Hands-On Practice

Now that your admin is set up, try these exercises:

### Exercise 1: Create Sample Data
1. Start the dev server: `python manage.py runserver`
2. Go to http://127.0.0.1:8000/admin/
3. Create 2-3 categories
4. Create 5-10 blog posts with different statuses

### Exercise 2: Test Admin Features
1. Use the search functionality
2. Try different filters
3. Use the date hierarchy
4. Test bulk actions (publish, draft, reset views)

### Exercise 3: Customize Further
1. Add a new field to Post model
2. Update admin to display the new field
3. Add a filter for the new field

### Exercise 4: Explore
1. Click on different posts to see the edit form
2. Try the horizontal filter for categories
3. Observe the auto-populated slug field

## 15. Troubleshooting

### Common Issues

**Issue**: Can't login to admin
**Solution**: Make sure you created a superuser with `python manage.py createsuperuser`

**Issue**: Models not showing in admin
**Solution**: Make sure you registered them in `admin.py` and the app is in `INSTALLED_APPS`

**Issue**: Admin is slow
**Solution**: Optimize queries with `select_related` and `prefetch_related` in `get_queryset`

**Issue**: Changes not showing
**Solution**: Restart the development server

**Issue**: "FieldError: Unknown field(s)" 
**Solution**: Check that all fields in `list_display`, `fields`, etc. exist in your model

## 16. Summary & Next Steps

### What We Accomplished

✅ Created a superuser account  
✅ Accessed the Django admin interface  
✅ Registered models with admin  
✅ Customized list displays with multiple columns  
✅ Added search functionality  
✅ Implemented filters and date hierarchy  
✅ Created custom display methods with formatting  
✅ Added bulk admin actions  
✅ Customized the admin site header and title  
✅ Optimized admin queries  

### Key Takeaways

1. Django admin is powerful and customizable
2. Basic registration is simple: `admin.site.register(Model)`
3. `ModelAdmin` class provides extensive customization
4. Custom methods can display computed or formatted data
5. Admin actions enable bulk operations
6. Performance matters - use query optimization


### Quick Check

Before moving on, ensure you can answer:

1. How do you register a model in the admin interface?
2. What is the purpose of ModelAdmin classes?
3. Name three ways to customize the admin list display
4. How do you add search functionality to the admin?

### What's Next

In **Module 04**, we'll:
- Create views to display data to users
- Learn about function-based views (FBVs)
- Explore class-based views (CBVs)
- Set up URL routing
- Handle HTTP requests and responses
- Work with generic views

---

**Great job! Your admin interface is now professional and user-friendly. Continue to Module 04!** 🚀